In [1]:
from hdx.hdx_configuration import Configuration 
from hdx.data.dataset import Dataset
import requests
import pandas as pd
import numpy as np
import csv
from zipfile import ZipFile
import os

In [2]:
def download_url(url, save_path, chunk_size=128):
    r = requests.get(url, stream=True)
    with open(save_path, 'wb') as fd:
        for chunk in r.iter_content(chunk_size=chunk_size):
            fd.write(chunk)

# Population

In [3]:
#url_pop = 'https://www.regions-et-departements.fr/fichiers/departements-francais.csv'
#download_url(url_pop, 'pop.csv')
#
#live = pd.read_csv('pop.csv', sep='\t')
#live.drop(index=[101,102], inplace=True)
#
#live
#
#live.columns = ['dep_num', 'name', 'region', 'capital', 'area', 'total', 'density']
#live['dep_num'] = live['dep_num'].replace({'2A':'201','2B':'202'}).astype(int)
#
#depts = pd.read_csv('../data/train/pop/fr/departements-francais.csv', sep=';')
#depts.columns = ['dep_num', 'name', 'region', 'capital', 'area', 'total', 'density']
#
#
#live == depts

# Covid

In [4]:
#import requests
#import pandas as pd
#import datetime
#departements = pd.read_csv("data/pop/fr/departements-francais.csv", sep = ";")
#hospi = []
#url = "https://coronavirusapi-france.now.sh/AllLiveData"
#response = requests.get(url).json()
#counter = 0
#for numero in departements.NUMÉRO:
#    nom = str(departements[departements["NUMÉRO"]==numero]["NOM"].values[0])
#    hospi.append((nom, numero, response["allLiveFranceData"][counter]["nouvellesHospitalisations"]))
#    counter+=1
#
#df = pd.DataFrame(hospi, columns =["depname","depnum","newhospi"])
#print(df)
#df.to_csv("covid_daily_data.csv", index = False)

# CAMS

In [5]:
#to_do (Niclas ?)

# Mobility

In [6]:
Configuration.create(hdx_site='prod', user_agent='A_Quick_Example', hdx_read_only=True)
dataset = Dataset.read_from_hdx('movement-range-maps')
resources = dataset.get_resources()
dic = resources[1]
url_mobility = dic['download_url']

file_mobility = "mvt_range.zip"
download_url(url_mobility, file_mobility)

In [7]:
with ZipFile(file_mobility, 'r') as zip:
    # printing all the contents of the zip file
    zip.printdir()
  
    # extracting all the files
    print('Extracting mv_range file now...')
    mvt_range = zip.namelist()[-1]
    zip.extract(mvt_range)
    print('Done!')

File Name                                             Modified             Size
README.txt                                     2021-04-15 10:48:02          961
movement-range-2021-04-14.txt                  2021-04-15 10:48:18    568169694
Extracting mv_range file now...
Done!


In [8]:
with open(mvt_range) as f:
    reader = csv.reader(f, delimiter="\t")
    d = list(reader)

In [9]:
data_mob = pd.DataFrame(d[1:], columns=d[0])
data_mob = data_mob[data_mob['country']=='FRA']

In [11]:
data_mob = data_mob[data_mob['ds']==list(data_mob.iloc[[-1]]['ds'])[0]][['ds', 'polygon_name', 'all_day_bing_tiles_visited_relative_change', 'all_day_ratio_single_tile_users']]

,ds,polygon_name,all_day_bing_tiles_visited_relative_change,all_day_ratio_single_tile_users
2396876,2021-04-14,Nouvelle-Aquitaine,-0.13703,0.20609
2397286,2021-04-14,Occitanie,-0.14055,0.20992
2397696,2021-04-14,Pays de la Loire,-0.14909,0.19821
2398106,2021-04-14,Provence-Alpes-Côte d'Azur,-0.16165,0.2216
2398516,2021-04-14,Auvergne-Rhône-Alpes,-0.11939,0.20583
2398926,2021-04-14,Bourgogne-Franche-Comté,-0.09288,0.21205
2399336,2021-04-14,Bretagne,-0.12397,0.19741
2399746,2021-04-14,Centre-Val de Loire,-0.111,0.21628
2400156,2021-04-14,Corse,-0.11303,0.19167
2400566,2021-04-14,Grand Est,-0.12887,0.2191


In [12]:
depts  = pd.read_csv('../data/train/pop/fr/departements-francais.csv', sep=';')
depts.columns = ['dep_num', 'name', 'region', 'capital', 'area', 'total', 'density']
depts = depts.sort_values('dep_num')
depts = depts[:-5]
depts['region'] = depts['region'].replace({'Ile-de-France':'Île-de-France'})
depts

,dep_num,name,region,capital,area,total,density
0,1,Ain,Auvergne-Rhône-Alpes,Bourg-en-Bresse,5762,631877,109.7
1,2,Aisne,Hauts-de-France,Laon,7369,538659,73.1
2,3,Allier,Auvergne-Rhône-Alpes,Moulins,7340,341613,46.5
3,4,Alpes-de-Haute-Provence,Provence-Alpes-Côte d'Azur,Digne,6925,161799,23.4
4,5,Hautes-Alpes,Provence-Alpes-Côte d'Azur,Gap,5549,140916,25.4
...,...,...,...,...,...,...,...
93,93,Seine-Saint-Denis,Île-de-France,Bobigny,236,1592663,6742.9
94,94,Val-de-Marne,Île-de-France,Créteil,245,1372389,5601.6
95,95,Val-d'Oise,Île-de-France,Pontoise,1246,1215390,975.5
19,201,Corse-du-Sud,Corse,Ajaccio,4014,152730,38.0


In [15]:
depts['stay_put'] = 0.0
depts['go_out'] = 0.0

In [27]:
def add_go_out(row):
    region = row['region']
    go_out = data_mob[data_mob['polygon_name']==region]['all_day_bing_tiles_visited_relative_change']
    return float(list(go_out)[0])
     
def add_stay_put(row):   
    region = row['region']
    stay_put = data_mob[data_mob['polygon_name']==region]['all_day_ratio_single_tile_users']
    return float(list(stay_put)[0])

In [29]:
depts['go_out'] = depts.apply(add_go_out, axis=1)
depts['stay_put'] = depts.apply(add_stay_put, axis=1)
depts

,dep_num,name,region,capital,area,total,density,relative_change,ratio,go_out,stay_put
0,1,Ain,Auvergne-Rhône-Alpes,Bourg-en-Bresse,5762,631877,109.7,-0.11939,0.0,-0.11939,0.20583
1,2,Aisne,Hauts-de-France,Laon,7369,538659,73.1,-0.13640,0.0,-0.13640,0.21761
2,3,Allier,Auvergne-Rhône-Alpes,Moulins,7340,341613,46.5,-0.11939,0.0,-0.11939,0.20583
3,4,Alpes-de-Haute-Provence,Provence-Alpes-Côte d'Azur,Digne,6925,161799,23.4,-0.16165,0.0,-0.16165,0.22160
4,5,Hautes-Alpes,Provence-Alpes-Côte d'Azur,Gap,5549,140916,25.4,-0.16165,0.0,-0.16165,0.22160
...,...,...,...,...,...,...,...,...,...,...,...
93,93,Seine-Saint-Denis,Île-de-France,Bobigny,236,1592663,6742.9,-0.17126,0.0,-0.17126,0.21999
94,94,Val-de-Marne,Île-de-France,Créteil,245,1372389,5601.6,-0.17126,0.0,-0.17126,0.21999
95,95,Val-d'Oise,Île-de-France,Pontoise,1246,1215390,975.5,-0.17126,0.0,-0.17126,0.21999
19,201,Corse-du-Sud,Corse,Ajaccio,4014,152730,38.0,-0.11303,0.0,-0.11303,0.19167


In [ ]:
def add_stay_put(row)

# Positive test